## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,80.33,73,20,10.36,few clouds
1,1,Sao Joao Da Barra,BR,-21.6403,-41.0511,68.07,73,99,6.87,overcast clouds
2,2,Hobart,AU,-42.8794,147.3294,72.01,54,40,1.01,scattered clouds
3,3,Tiksi,RU,71.6872,128.8694,-16.29,99,98,0.94,overcast clouds
4,4,Dunedin,NZ,-45.8742,170.5036,52.54,82,100,4.74,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hobart,AU,-42.8794,147.3294,72.01,54,40,1.01,scattered clouds
10,10,Nouadhibou,MR,20.9310,-17.0347,70.59,72,100,11.01,overcast clouds
11,11,Omboue,GA,-1.5746,9.2618,78.01,91,100,3.87,overcast clouds
12,12,Itarema,BR,-2.9248,-39.9167,79.30,79,22,12.41,few clouds
15,15,Carnarvon,AU,-24.8667,113.6333,75.27,43,0,19.57,clear sky
23,23,Bongaree,AU,-27.0797,153.1586,78.30,78,52,11.25,light rain
24,24,Adrar,MR,20.5022,-10.0711,76.59,29,100,10.02,overcast clouds
35,35,New Norfolk,AU,-42.7826,147.0587,71.65,37,36,1.99,scattered clouds
37,37,Rikitea,PF,-23.1203,-134.9692,74.50,61,3,4.38,clear sky
46,46,Manaus,BR,-3.1019,-60.0250,77.49,94,0,2.30,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hobart,AU,72.01,scattered clouds,-42.8794,147.3294,
10,Nouadhibou,MR,70.59,overcast clouds,20.9310,-17.0347,
11,Omboue,GA,78.01,overcast clouds,-1.5746,9.2618,
12,Itarema,BR,79.30,few clouds,-2.9248,-39.9167,
15,Carnarvon,AU,75.27,clear sky,-24.8667,113.6333,
23,Bongaree,AU,78.30,light rain,-27.0797,153.1586,
24,Adrar,MR,76.59,overcast clouds,20.5022,-10.0711,
35,New Norfolk,AU,71.65,scattered clouds,-42.7826,147.0587,
37,Rikitea,PF,74.50,clear sky,-23.1203,-134.9692,
46,Manaus,BR,77.49,clear sky,-3.1019,-60.0250,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hobart,AU,72.01,scattered clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
10,Nouadhibou,MR,70.59,overcast clouds,20.9310,-17.0347,El Medina
11,Omboue,GA,78.01,overcast clouds,-1.5746,9.2618,Hotel Olako
12,Itarema,BR,79.30,few clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
15,Carnarvon,AU,75.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
23,Bongaree,AU,78.30,light rain,-27.0797,153.1586,The Gums Anchorage
35,New Norfolk,AU,71.65,scattered clouds,-42.7826,147.0587,The Shingles Riverside Cottages
37,Rikitea,PF,74.50,clear sky,-23.1203,-134.9692,People ThankYou
46,Manaus,BR,77.49,clear sky,-3.1019,-60.0250,Manaus Hotéis Millennium
54,Adilabad,IN,75.31,overcast clouds,19.6667,78.5333,Dwaraka Hotel.


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))